# Task 3: Baseline Model Comparison (Neural Network)

In this notebook, you will train a simple Neural Network (MLP) to predict the `prior_hiring_decision` target variable. Compare its performance to the previous models.

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Load Data
train_df = pd.read_csv('../data/train.csv')
val_df = pd.read_csv('../data/val.csv')
test_df = pd.read_csv('../data/test.csv')

columns_to_drop = ["Age", "Sex", "Race", "Place_Of_Birth", "Hours_Per_Week", "Marital_Status", "Relationship"]

for df in [train_df, val_df, test_df]:
    df.drop(columns=columns_to_drop, inplace=True)
print("Training Shape:", train_df.shape)

Training Shape: (30000, 6)


## 1. Preprocessing

In [ ]:
import torch
import numpy as np

# Define target and features
target = 'prior_hiring_decision'
features = [col for col in train_df.columns if col != target]

X_train = train_df[features]
y_train = train_df[target]
X_val = val_df[features]
y_val = val_df[target]
X_test = test_df[features]
y_test = test_df[target]

# All features are numeric after previous drops; scale them
numerical_cols = features
preprocessor = StandardScaler()

# Preprocess data
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)

# PyTorch classifier with sklearn-like API
import torch.nn as nn
import torch.optim as optim

class TorchBinaryMLP:
    def __init__(self,
                 hidden_layer_sizes=(64,),
                 alpha=0.0001,
                 learning_rate_init=0.001,
                 max_epochs=500,
                 random_state=42,
                 early_stopping=True,
                 batch_size=256,
                 patience=10,
                 device=None):
        self.hidden_layer_sizes = hidden_layer_sizes
        self.alpha = alpha
        self.learning_rate_init = learning_rate_init
        self.max_epochs = max_epochs
        self.random_state = random_state
        self.early_stopping = early_stopping
        self.batch_size = batch_size
        self.patience = patience
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.model_ = None
        self.input_dim_ = None

    def _build_model(self, input_dim):
        layers = []
        prev = input_dim
        for h in self.hidden_layer_sizes:
            layers.append(nn.Linear(prev, h))
            layers.append(nn.ReLU())
            prev = h
        layers.append(nn.Linear(prev, 1))
        self.model_ = nn.Sequential(*layers).to(self.device)

    def fit(self, X, y):
        rng = np.random.RandomState(self.random_state)
        torch.manual_seed(self.random_state)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(self.random_state)

        X = np.asarray(X, dtype=np.float32)
        y = np.asarray(y, dtype=np.float32).reshape(-1, 1)

        self.input_dim_ = X.shape[1]
        self._build_model(self.input_dim_)

        optimizer = optim.Adam(self.model_.parameters(), lr=self.learning_rate_init, weight_decay=self.alpha)
        criterion = nn.BCEWithLogitsLoss()

        # internal validation split for early stopping
        if self.early_stopping:
            idx = rng.permutation(len(X))
            split = int(len(X) * 0.9)
            train_idx, val_idx = idx[:split], idx[split:]
            X_tr, y_tr = X[train_idx], y[train_idx]
            X_va, y_va = X[val_idx], y[val_idx]
        else:
            X_tr, y_tr = X, y
            X_va, y_va = None, None

        X_tr_t = torch.from_numpy(X_tr).to(self.device)
        y_tr_t = torch.from_numpy(y_tr).to(self.device)

        best_val_loss = float('inf')
        best_state = None
        patience_left = self.patience

        for epoch in range(self.max_epochs):
            self.model_.train()
            # mini-batch training
            perm = rng.permutation(len(X_tr_t))
            for i in range(0, len(X_tr_t), self.batch_size):
                batch_idx = perm[i:i+self.batch_size]
                xb = X_tr_t[batch_idx]
                yb = y_tr_t[batch_idx]
                optimizer.zero_grad()
                logits = self.model_(xb)
                loss = criterion(logits, yb)
                loss.backward()
                optimizer.step()

            if self.early_stopping and X_va is not None:
                self.model_.eval()
                with torch.no_grad():
                    X_va_t = torch.from_numpy(X_va).to(self.device)
                    y_va_t = torch.from_numpy(y_va).to(self.device)
                    val_logits = self.model_(X_va_t)
                    val_loss = criterion(val_logits, y_va_t).item()

                if val_loss < best_val_loss - 1e-5:
                    best_val_loss = val_loss
                    best_state = {k: v.clone() for k, v in self.model_.state_dict().items()}
                    patience_left = self.patience
                else:
                    patience_left -= 1
                    if patience_left <= 0:
                        break

        if best_state is not None:
            self.model_.load_state_dict(best_state)

        return self

    def predict_proba(self, X):
        X = np.asarray(X, dtype=np.float32)
        self.model_.eval()
        with torch.no_grad():
            X_t = torch.from_numpy(X).to(self.device)
            logits = self.model_(X_t)
            probs_pos = torch.sigmoid(logits).cpu().numpy().ravel()
        probs_neg = 1.0 - probs_pos
        return np.column_stack([probs_neg, probs_pos])

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)

    def score(self, X, y):
        y_pred = self.predict(X)
        y = np.asarray(y)
        return (y_pred == y).mean()

    def get_params(self, deep=True):
        return {
            'hidden_layer_sizes': self.hidden_layer_sizes,
            'alpha': self.alpha,
            'learning_rate_init': self.learning_rate_init,
            'max_epochs': self.max_epochs,
            'random_state': self.random_state,
            'early_stopping': self.early_stopping,
            'batch_size': self.batch_size,
            'patience': self.patience,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self

# Hyperparameter grid
param_grid = {
    'hidden_layer_sizes': [(64,), (128,), (64, 32), (128, 64)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01]
}

best_score = 0.0
best_params = {}

for hidden in param_grid['hidden_layer_sizes']:
    for alpha in param_grid['alpha']:
        for lr in param_grid['learning_rate_init']:
            clf = TorchBinaryMLP(
                hidden_layer_sizes=hidden,
                alpha=alpha,
                learning_rate_init=lr,
                max_epochs=500,
                random_state=42,
                early_stopping=True
            )
            clf.fit(X_train_processed, y_train)
            score = clf.score(X_val_processed, y_val)
            if score > best_score:
                best_score = score
                best_params = {'hidden_layer_sizes': hidden, 'alpha': alpha, 'learning_rate_init': lr}

print(f"Best params: {best_params}")
print(f"Best validation accuracy: {best_score:.4f}")

# Train final model
best_clf = TorchBinaryMLP(**best_params, max_epochs=500, random_state=42, early_stopping=True)
best_clf.fit(X_train_processed, y_train)

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', best_clf)
])

## 2. Preprocessing Fairness

In [ ]:
def compute_intersectional_weights(df, protected_cols, label_col):
    total = len(df)
    labels = df[label_col].unique()
    
    df = df.copy()
    df['_group'] = df[protected_cols].astype(str).agg('_'.join, axis=1)
    groups = df['_group'].unique()
    
    weights = {}
    for group in groups:
        group_count = len(df[df['_group'] == group])
        for label in labels:
            label_count = len(df[df[label_col] == label])
            intersection_count = len(df[(df['_group'] == group) & (df[label_col] == label)])
            
            if intersection_count > 0:
                weight = (group_count * label_count) / (total * intersection_count)
            else:
                weight = 1.0
            
            weights[(group, int(label))] = float(weight)
    
    return weights, df['_group']

def compute_sample_weights(df, protected_cols, label_col):
    group_weights, group_col = compute_intersectional_weights(df, protected_cols, label_col)
    
    sample_weights = []
    for idx, row in df.iterrows():
        group_key = '_'.join([str(row[col]) for col in protected_cols])
        label = int(row[label_col])
        weight = group_weights.get((group_key, label), 1.0)
        sample_weights.append(weight)
    
    return np.array(sample_weights), group_weights

protected_cols = ['Sex', 'Race']
sample_weights, group_weights = compute_sample_weights(train_df, protected_cols, target)

print("Intersectional Group Weights:")
for (group, label), weight in sorted(group_weights.items()):
    print(f"  Group={group}, Label={label}: {weight:.4f}")

print(f"\nSample weights computed for {len(train_df)} samples")
print(f"Weight range: [{sample_weights.min():.4f}, {sample_weights.max():.4f}]")

## 3. Training

## 4. Evaluation

## 5. Fairness Metric Implementation

In [ ]:
%run ../other_files/task2.ipynb

protected_characteristics = ['Sex', 'Race', 'Age']

val_with_preds = val_df.copy()
val_with_preds['prediction'] = best_clf.predict(X_val_processed)

def compute_fairness_metrics(data, protected_characteristics, target='prediction'):
    results = {
        'demographic_parity': {},
        'equalized_opportunity': {},
        'equalized_odds': {}
    }
    
    for char in protected_characteristics:
        groups = data[char].unique()
        
        dp_info = {}
        for group in groups:
            group_data = data[data[char] == group]
            dp_info[int(group)] = {
                'positive_rate': float(group_data[target].mean()),
                'count': int(len(group_data))
            }
        dp_info['ratio'] = float(demographic_parity(char, target, data))
        results['demographic_parity'][char] = dp_info
        
        eo_info = {}
        for group in groups:
            group_data = data[data[char] == group]
            positive_cases = group_data[group_data[target] == 1]
            eo_info[int(group)] = {
                'tpr': float(positive_cases[target].mean()) if len(positive_cases) > 0 else None,
                'positive_count': int(len(positive_cases))
            }
        eo_info['ratio'] = float(equalized_opportunity(char, target, data))
        results['equalized_opportunity'][char] = eo_info
        
        eod_info = {}
        for group in groups:
            group_data = data[data[char] == group]
            positive_cases = group_data[group_data[target] == 1]
            negative_cases = group_data[group_data[target] == 0]
            eod_info[int(group)] = {
                'tpr': float(positive_cases[target].mean()) if len(positive_cases) > 0 else None,
                'fpr': float(negative_cases[target].mean()) if len(negative_cases) > 0 else None,
            }
        eod_info['ratio'] = float(equalized_odds(char, target, data))
        results['equalized_odds'][char] = eod_info
    
    return results

fairness_results = compute_fairness_metrics(val_with_preds, protected_characteristics, 'prediction')
print("Fairness Metrics (Neural Network):")
print("\nDemographic Parity:", fairness_results['demographic_parity'])
print("\nEqualized Opportunity:", fairness_results['equalized_opportunity'])
print("\nEqualized Odds:", fairness_results['equalized_odds'])

## 6. ROC Curve Analysis

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

y_proba = best_clf.predict_proba(X_val_processed)[:, 1]
fpr, tpr, thresholds = roc_curve(y_val, y_proba)
auc_score = roc_auc_score(y_val, y_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Neural Network (AUC = {auc_score:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Neural Network')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

print(f"AUC Score: {auc_score:.4f}")

## 7. Adjusted Equalized Odds

In [ ]:
def adjusted_equalized_odds(data, protected_col, y_true_col, y_proba, thresholds):
    """
    Calculate equalized odds using group-specific classification thresholds.
    
    Args:
        data: DataFrame with protected characteristic and true labels
        protected_col: Name of protected characteristic column
        y_true_col: Name of true label column
        y_proba: Array of predicted probabilities
        thresholds: Dict mapping group values to classification thresholds
    """
    data = data.copy()
    data['y_proba'] = y_proba
    
    data['adjusted_pred'] = data.apply(
        lambda row: 1 if row['y_proba'] >= thresholds.get(row[protected_col], 0.5) else 0,
        axis=1
    )
    
    groups = data[protected_col].unique()
    tpr_rates = []
    fpr_rates = []
    results = {}
    
    for group in groups:
        group_data = data[data[protected_col] == group]
        positive_cases = group_data[group_data[y_true_col] == 1]
        negative_cases = group_data[group_data[y_true_col] == 0]
        
        tpr = positive_cases['adjusted_pred'].mean() if len(positive_cases) > 0 else 0
        fpr = negative_cases['adjusted_pred'].mean() if len(negative_cases) > 0 else 0
        
        tpr_rates.append(tpr)
        fpr_rates.append(fpr)
        
        results[int(group)] = {
            'tpr': float(tpr),
            'fpr': float(fpr),
            'threshold': float(thresholds.get(group, 0.5)),
            'count': int(len(group_data))
        }
    
    max_tpr, min_tpr = max(tpr_rates), min(tpr_rates)
    max_fpr, min_fpr = max(fpr_rates), min(fpr_rates)
    
    tpr_ratio = max_tpr / min_tpr if min_tpr > 0 else float('inf')
    fpr_ratio = max_fpr / min_fpr if min_fpr > 0 else float('inf')
    
    results['tpr_ratio'] = float(tpr_ratio)
    results['fpr_ratio'] = float(fpr_ratio)
    results['equalized_odds_ratio'] = float(max(tpr_ratio, fpr_ratio))
    
    return results